In [ ]:
from hydromt_sfincs import SfincsModel, utils
import numpy as np
from os.path import join
import hydromt
import glob
import pandas as pd
from datetime import timedelta
import xarray as xr
import os
import matplotlib.pyplot as plt

## read flood and model data

In [ ]:
mdir = r"../../3_models/sfincs"
rdir = r"../../4_results"

In [ ]:
mod0 = SfincsModel(join(mdir, '00_base_riv'), mode='r')
rivmsk = mod0.staticmaps['rivmsk'].raster.flipud()==1

In [ ]:
index_cols = ['h_rp','p_rp','qb_rp','qp_rp']
ds = xr.open_zarr(join(rdir, 'hmax.zarr')).swap_dims({'index':'scen'}).set_coords(index_cols)
da = ds['hmax'].fillna(0)
da.attrs.update(_FillValue=da.attrs['nodatavals'][0])

## Bias correct

In [ ]:
# bias correct for Q and H but make sure to keep P
nodata = da.attrs['_FillValue']
# get Q2 + MSHW map for bias correction
bias, da0 = 'q2', da.sel(scen=['qb002_qp002_h000_p000']).squeeze(drop=True)
mask = np.logical_and(da0!=nodata, ~rivmsk)
# account for minimal P in all scenarios
pscen = [f'qb000_qp000_h000_p{s[-3:]}' for s in da.scen.values]
dap = da.sel(scen=pscen).reset_coords(drop=True).drop('scen')
dap = np.maximum(0, dap - da.sel(scen=['qb000_qp000_h000_p000']).squeeze(drop=True))
# take maximum of pscen and bias corrected scens
da1 = np.maximum(dap, da-da0).where(mask, 0)
# da1.attrs.update(**da.attrs)
# da1 = da1.chunk({'x':-1, 'y':-1, 'scen':10})

## calculate compound flood depths based on 5000 yrs sample

In [ ]:

df_sample = pd.read_csv(join(rdir, r'sim_EVENTS_rp.csv'), index_col=0).rename(columns={'h_tsw_rp': 'h_rp', 'h_tsw': 'h'})


df_flood = da1.reset_coords().drop(['band', 'spatial_ref', 'x', 'y', 'hmax']).to_dataframe()
# df_flood[index_cols] = np.maximum(1, df_flood[index_cols])
ds_flood_rp = df_flood.reset_index().drop(columns=index_cols).set_index(
    pd.MultiIndex.from_frame(df_flood[index_cols])
).to_xarray()
ds_flood_rp['index'] = ds_flood_rp['index'].fillna(-1).astype(int)
ds_flood_rp


In [ ]:
df_sim_events1 = pd.read_csv(join(rdir, r'sim_EVENTS_rp.csv'), index_col=0)#.rename(columns={'h_tsw_rp': 'h_rp', 'h_tsw': 'h'})

# get all scenarios used for linear interpolating damages
rps = np.array([  0,  2,   5,  10,  25,  50, 100, 250, 500])
cols = ['qb_rp', 'qp_rp', 'p_rp', 'h_tsw_rp']
print(len(rps)**len(cols))
scens = []

values = df_sim_events1.loc[:,cols].values
for event in values:
    rps_lst = []
    for rp in event:
        if rp in rps:
            rps_lst.append([int(rp), int(rp)])
        else:
            rps_lst.append([rps[rps<=rp][-1], rps[rps>=rp][0]])
    for rps0 in zip(*rps_lst):
        scens.append(list(rps0))
# include univariate
rps0 = np.zeros(4, dtype=int)
for i in range(4):
    for rp in rps[1:]:
        _rps = rps0.tolist()
        _rps[i] = rp
        scens.append(_rps)
# include full dependence
for rp in rps:
    scens.append(np.full(4, rp, dtype=int).tolist())
##
df_scen = pd.DataFrame(data=np.vstack(scens), columns=cols).value_counts().rename('count').reset_index()
df_scen['scen'] = [
    f"qb{qb_rp:03.0f}_qp{qp_rp:03.0f}_h{h_rp:03.0f}_p{p_rp:03.0f}" 
    for i, (qb_rp, qp_rp, p_rp, h_rp) in df_scen[cols].iterrows()
]
df_scen
print(df_scen.index.size)
# df_scen.to_csv(r'../../4_results/sim_SCEN.csv')
df_scen.sort_values('count', ascending=False).head(10)

In [ ]:
# df_scen.set_index('scen').loc
['qb005_qp050_h500_p002'] in df_scen['scen'].values.tolist()

In [ ]:
a0 = ds_flood_rp.sel(ds0_rp, method='nearest')
a0.where(a0['index']<0, drop=True)

In [ ]:
# get nearest scen for each sample
ds0_rp = df_sample[index_cols + ['year']].to_xarray().set_coords('year')
df_sample['index'] = ds_flood_rp['index'].sel(ds0_rp, method='nearest')
# df_sample['scen'] = df_flood.reset_index().loc[df_sample['index'].values, 'scen'].values

df_sample[df_sample['index']<0]

In [ ]:
_scen_unique, idxs_reverse = np.unique(df_sample['scen'], return_inverse=True)
print(_scen_unique.size)
flddph = da1.sel(scen=_scen_unique).load()

In [ ]:
from dask import delayed
import dask
rps=np.array([2, 5, 10, 50, 100, 500])
cdf = np.arange(5000)/5000
bins = df_sample[['year']].values.flatten()
def _fldsrt(fld0, idxs_reverse=idxs_reverse, rp=rps, bins=bins, cdf=cdf):
    if np.all(fld0==0):
        return 0.0
    fld0 = fld0[idxs_reverse]
    df0 = pd.DataFrame(data={'fld0': fld0, 'bin':bins})
    fld_max = df0[['fld0', 'bin']].groupby('bin').max().values.flatten()
    fld_sorted = np.sort(fld_max)
    return np.interp(1-1/rp, cdf, fld_sorted)
data=np.apply_along_axis(_fldsrt, flddph.get_axis_num("scen"), flddph)

In [ ]:
da_cmpnd = xr.DataArray(
    data=data, 
    dims=('rp', 'y', 'x'),
    coords={'rp':rps, **da1.raster.coords}
)
da_cmpnd.raster.set_nodata(da1.raster.nodata)
da_cmpnd.raster.set_crs(da1.raster.crs)
for rp0 in rps:
    da_cmpnd.sel(rp=rp0).raster.to_raster(join(r'../03_models/hmax', f'compound_{rp0:03d}.tif'), compress='lzw')

## flood impact assessment

In [ ]:
# log-linear integration from FIAT
def get_logcov(rp_lst):
    f_lst = [1 / i for i in rp_lst]
    lf = [np.log(1 / i) for i in rp_lst]
    c = [(1 / (lf[i] - lf[i+1])) for i in range(len(rp_lst[:-1]))]
    G = [(f_lst[i] * lf[i] - f_lst[i]) for i in range(len(rp_lst))]
    a = [((1 + c[i] * lf[i+1]) * (f_lst[i] - f_lst[i+1]) + c[i] * (G[i+1] - G[i])) for i in range(len(rp_lst[:-1]))]
    b = [(c[i] * (G[i] - G[i+1] + lf[i+1] * (f_lst[i+1] - f_lst[i]))) for i in range(len(rp_lst[:-1]))]
    if len(rp_lst) == 1:
        cov_lst = f_lst
    else:
        cov_lst = [b[0] if i == 0 else f_lst[i] + a[i-1] if i == len(rp_lst) - 1 else a[i-1] + b[i] for i in range(len(rp_lst))]

    return cov_lst

def loglin_trapz(exp, rp_lst=None):
    if rp_lst is None:
        rp_lst = (1/(1-np.arange(exp.size)/exp.size)).tolist()
    exp_lst = np.sort(exp).tolist()
    cov_lst = get_logcov(rp_lst)
    risk = (np.asarray(cov_lst) * np.asarray(exp_lst)).sum()
    return risk

In [ ]:
hmin = 0.15

# small edit to set threshold for impact at 15cm
df = pd.read_csv(join(mdir, '98_fiat', 'susceptibility', 'AF000.csv'), index_col=0)
df.columns = ['factor']
df.index.name = 'depth'
df[df.index<hmin] = 0
ds_exp = hydromt.open_mfraster(join(mdir, '98_fiat', 'exposure', '*.tif')).load()

In [ ]:
samples = dict()
for postfix in ['_magInd_timeObs', '_magInd_timeInd', '_magObs_timeObs']:
    fn = os.path.join('../02_data', f'modelled_dataset_5000_years{postfix}_rp.csv')
    samples[postfix[1:]] = pd.read_csv(fn, index_col=0)

In [ ]:

def flood_damage(da_flddph, da_exposure, df_susceptibility, **kwargs):
    nodata = da_exposure.attrs['_FillValue']
    da0 = df_susceptibility.to_xarray()['factor'].chunk({'depth':-1})
    factor = np.minimum(1, da0.interp(depth=da_flddph, **kwargs))
    damage = (factor * da_exposure).fillna(nodata).astype(np.float32)
    damage.name = da_exposure.name
    damage.attrs.update(**da_exposure.attrs)
    return damage

def flood_exposed(da_flddph, da_exposure, min_flddph=hmin):
    exposed = xr.where(da_flddph>min_flddph,da_exposure,0.0).astype(np.float32)
    exposed.attrs.update(**da_exposure.attrs)
    exposed.name = da_exposure.name
    return exposed



## base scenario

In [ ]:
# check for single scen
# scen='qb100_qp100_h000_p000'
# da_dam = flood_damage(da1.sel(scen=scen), da_exp0, df).compute()
# da_ppl = flood_exposed(da1.sel(scen=scen), da_exp1, 0.15).compute()
# print(da_dam.sum().item()/1e6)
# print(da_ppl.sum().item()/1e3)

# root = glob.glob(join(mdir, f'*{scen}'))[0]
# print(root)
# da1.sel(scen=scen).raster.to_raster(join(root, 'gis', 'hmax_bias.tif'), compress='lzw')
# fn_out = join(root, 'gis', f'{exposure}.tif') 
# da_dam.raster.to_raster(fn_out, compress='lzw')
# fn_out = join(root, 'gis', f'population_count.tif') 
# da_ppl.raster.to_raster(fn_out, compress='lzw')

In [ ]:
rps = np.array([0,2,5,10,50,100,500], dtype=int)
fns = os.path.join('../03_models', 'hmax', f'*.tif') 
ds = hydromt.open_mfraster(fns)
da_qb = xr.concat([ds[f'qb{rp:03d}_qp000_h000_p000'] for rp in rps], dim='rps').rename('qb')
da_qp = xr.concat([ds[f'qb000_qp{rp:03d}_h000_p000'] for rp in rps], dim='rps').rename('qp')
da_h = xr.concat([ds[f'qb000_qp000_h{rp:03d}_p000'] for rp in rps], dim='rps').rename('h')
da_p = xr.concat([ds[f'qb000_qp000_h000_p{rp:03d}'] for rp in rps], dim='rps').rename('p')
da_c = xr.concat([da_p.isel(rps=0)] + [ds[f'compound_{rp:03d}'] for rp in rps[1:]], dim='rps').rename('cp')
ds = xr.merge([da_qb, da_qp, da_h, da_p, da_c])
rps[0] = 1
ds['rps'] = xr.IndexVariable('rps', rps)
ds['cof'] = xr.IndexVariable('rps', get_logcov(rps))
ds = ds.set_coords('cof').load()
ds_impact = xr.Dataset()
for dvar in ds.data_vars:
    ds_impact[f'{dvar}_dam'] = flood_damage(ds[dvar], ds_exp['buildings_value'], df)
    ds_impact[f'{dvar}_pop'] = flood_exposed(ds[dvar], ds_exp['population_count'], hmin)

In [ ]:
da_dam = flood_damage(da1, ds_exp['buildings_value'], df)
da_dam.attrs.update(unit='USD')
da_ppl = flood_exposed(da1, ds_exp['population_count'], hmin)
da_dam.attrs.update(unit='people')

In [ ]:
fn = os.path.join('../02_data', f'modelled_dataset_5000_years_magObs_timeObs_scenCount.csv')
df_scen = pd.read_csv(fn, index_col=0)
ds_impact = xr.merge([da_dam, da_ppl])#.sel(scen=df_scen['scen'].values)
ds_impact.chunk({'x':-1, 'y':-1, 'scen':10}).to_zarr(join(mdir, 'flood_impact.zarr'))

In [ ]:
da_dam_agg = da_dam.sum(('x', 'y')).round(2)
da_ppl_agg = da_ppl.sum(('x', 'y')).round(2)
df_out = da_dam_agg.reset_coords().drop_vars(['band','spatial_ref']).to_dataframe()
df_out[da_ppl.name] = da_ppl_agg.reset_coords(drop=True).to_dataframe()
df_out.to_csv(join(mdir, '98_fiat', 'flood_impact_base.csv'))

In [ ]:
index_cols = ['h_rp','p_rp','qb_rp','qp_rp']
drop_cols = ['scen']#, 'h','p','qb','qp']
exp_cols = ['buildings_value', 'population_count']

# read impacts and convert to 4D xarray
df_flood = pd.read_csv(join(mdir, '98_fiat', 'flood_impact_base.csv'), index_col=0)
df_flood[index_cols] = np.maximum(1, df_flood[index_cols])
df_flood = df_flood.reset_index()
ds_flood_rp = df_flood.reset_index().drop(columns=index_cols+drop_cols).set_index(
    pd.MultiIndex.from_frame(df_flood[index_cols])
).to_xarray()

# interpolate damages for each sample
for key in samples:
    ds0_rp = samples[key][index_cols + ['year']].to_xarray().set_coords('year')
    ds0_flood = ds_flood_rp[cols].interp(ds0_rp, method='linear')
    for col in cols:
        samples[key][f'{col}_base'] = ds0_flood[col].to_series()#.fillna(0)

## dikes rp 10

In [ ]:
qrp, hrp = 10, 10

# interpolate damages for each sample
for key in samples:
    df0 = samples[key][index_cols + ['year']].copy()
    df0['h_rp'] = np.where(df0['h_rp']<hrp,1,df0['h_rp'])
    df0['qb_rp'] = np.where(df0['qb_rp']<hrp,1,df0['qb_rp'])
    df0['qp_rp'] = np.where(df0['qp_rp']<hrp,1,df0['qp_rp'])
    ds0_rp = df0.to_xarray().set_coords('year')
    ds0_flood = ds_flood_rp[cols].interp(ds0_rp, method='linear')
    for col in cols:
        samples[key][f'{col}_dikes'] = ds0_flood[col].to_series()#.fillna(0)

In [ ]:
# kort door de bocht!! 
ds_impact[f'cp_dam_dikes'] = ds_impact[f'cp_dam'].sel(rps=[10, 50, 100, 500])
ds_impact[f'cp_pop_dikes'] = ds_impact[f'cp_pop'].sel(rps=[10, 50, 100, 500])

## zoning rp 5

In [ ]:
# correct exposure with 5 year flood zone
qrp, hrp = 5, 5
da0 = da1.sel(scen=[f'qb{qrp:03d}_qp000_h000_p000', f'qb000_qp{qrp:03d}_h000_p000', f'qb000_qp000_h{hrp:03d}_p000']).max('scen').squeeze(drop=True)
fld_bin = da0 > hmin
ds_exp_zone = ds_exp.where(~fld_bin, 0)

(ds_exp - ds_exp_zone).sum().compute()

In [ ]:
da_dam = flood_damage(da1, ds_exp_zone['buildings_value'], df)
da_dam_agg = da_dam.sum(('x', 'y')).round(2)
da_ppl = flood_exposed(da1, ds_exp_zone['population_count'], hmin)
da_ppl_agg = da_ppl.sum(('x', 'y')).round(2)

df_out = da_dam_agg.reset_coords().drop_vars(['band','spatial_ref']).to_dataframe()
df_out[da_ppl.name] = da_ppl_agg.reset_coords(drop=True).to_dataframe()
df_out.to_csv(join(mdir, '98_fiat', 'flood_impact_zoning.csv'))

In [ ]:
# read impacts and convert to 4D xarray
df_flood = pd.read_csv(join(mdir, '98_fiat', 'flood_impact_zoning.csv'), index_col=0)
df_flood[index_cols] = np.maximum(1, df_flood[index_cols])
df_flood = df_flood.reset_index()
ds_flood_rp = df_flood.reset_index().drop(columns=index_cols+drop_cols).set_index(
    pd.MultiIndex.from_frame(df_flood[index_cols])
).to_xarray()

# interpolate damages for each sample
for key in samples:
    ds0_rp = samples[key][index_cols + ['year']].to_xarray().set_coords('year')
    ds0_flood = ds_flood_rp[cols].interp(ds0_rp, method='linear')
    for col in cols:
        samples[key][f'{col}_zoning'] = ds0_flood[col].to_series()#.fillna(0)

In [ ]:
ds_impact[f'cp_dam_zoning'] = flood_damage(ds['cp'], ds_exp_zone['buildings_value'], df)
ds_impact[f'cp_pop_zoning'] = flood_exposed(ds['cp'], ds_exp_zone['population_count'], hmin)

## dry proofing

In [ ]:
# modify depth-damage curve
hmin_dry = 0.5
df_dry = df.copy()
df_dry[df_dry.index<hmin_dry] = 0

In [ ]:
da_dam = flood_damage(da1, ds_exp['buildings_value'], df_dry)
da_dam_agg = da_dam.sum(('x', 'y')).round(2)
da_ppl = flood_exposed(da1, ds_exp['population_count'], hmin_dry)
da_ppl_agg = da_ppl.sum(('x', 'y')).round(2)

df_out = da_dam_agg.reset_coords().drop_vars(['band','spatial_ref']).to_dataframe()
df_out[da_ppl.name] = da_ppl_agg.reset_coords(drop=True).to_dataframe()
df_out.to_csv(join(mdir, '98_fiat', 'flood_impact_dryproofing.csv'))

In [ ]:
# read impacts and convert to 4D xarray
df_flood = pd.read_csv(join(mdir, '98_fiat', 'flood_impact_dryproofing.csv'), index_col=0)
df_flood[index_cols] = np.maximum(1, df_flood[index_cols])
df_flood = df_flood.reset_index()
ds_flood_rp = df_flood.reset_index().drop(columns=index_cols+drop_cols).set_index(
    pd.MultiIndex.from_frame(df_flood[index_cols])
).to_xarray()

# interpolate damages for each sample
for key in samples:
    ds0_rp = samples[key][index_cols + ['year']].to_xarray().set_coords('year')
    ds0_flood = ds_flood_rp[cols].interp(ds0_rp, method='linear')
    for col in cols:
        samples[key][f'{col}_dryproofing'] = ds0_flood[col].to_series()#.fillna(0)

In [ ]:
for key in samples:
    df_sample = samples[key]
    fn = os.path.join('../02_data', f'modelled_dataset_5000_years_{key}_rp_impact.csv')
    df_sample.to_csv(fn)

In [ ]:
ds_impact[f'cp_dam_dryproofing'] = flood_damage(ds['cp'], ds_exp['buildings_value'], df_dry)
ds_impact[f'cp_pop_dryproofing'] = flood_exposed(ds['cp'], ds_exp['population_count'], hmin_dry)

In [ ]:
# # combine outputs
# cols = ['buildings_value', 'population_count']
# drop_cols = ['h_rp','p_rp','qb_rp','qp_rp']
# df_lst = []
# for postfix in ['base', 'dikes', 'dryproofing', 'zoning']:
#     dfi = pd.read_csv(join(mdir, '98_fiat', f'flood_impact_{postfix}.csv'), index_col=0)
#     dfi = dfi.rename(columns={c: f'{c}_{postfix}' for c in cols})
#     if postfix != 'base':
#         dfi = dfi.drop(columns=drop_cols)
#     df_lst.append(dfi)
# df_out = pd.concat(df_lst, axis=1)
# df_out.to_csv(join(mdir, '98_fiat', 'flood_impact.csv'))

## risk calculations

In [ ]:
samples = dict()
index_cols = ['h_rp','p_rp','qb_rp','qp_rp']
exp_cols = ['buildings_value','population_count']
for postfix in ['_magInd_timeObs', '_magInd_timeInd', '_magObs_timeObs']:
    fn = os.path.join('../02_data', f'modelled_dataset_5000_years{postfix}_rp_impact.csv')
    samples[postfix[1:]] = pd.read_csv(fn, index_col=0)
    
df_flood = pd.read_csv(join(mdir, '98_fiat', 'flood_impact_base.csv'), index_col=0)
df_flood = df_flood.rename(columns={c: f'{c}_base' for c in exp_cols})
df_flood[index_cols] = np.maximum(1, df_flood[index_cols])

In [ ]:
exp_cols0 = df_flood.columns[-2:].values.tolist()
risk = dict()
dfs = dict()
for col in index_cols:
    zero_cols = [c for c in index_cols if c != col]
    df0 = df_flood[np.all(df_flood[zero_cols]==1, axis=1)].sort_values(col)
    risk[col.split('_')[0]] = pd.Series({c: loglin_trapz(df0[c], df0[col]) for c in exp_cols0})
    dfs[col.split('_')[0]] = df0[[col] + exp_cols0].rename(columns={col: 'rp'})
sum([df[exp_cols0[0]] for _, df in risk.items()])

In [ ]:
df0 = df_flood[np.all(np.diff(df_flood[index_cols], axis=1)==0, axis=1)].sort_values(col)
dfs['fullDep'] = df0[['h_rp'] + exp_cols0].rename(columns={'h_rp': 'rp'})
risk['fullDep'] = pd.Series({c: loglin_trapz(df0[c], df0['h_rp']) for c in exp_cols0})

In [ ]:
for key in samples:
    df = samples[key]
    
    exp_cols = [c for c in df.columns if (c.startswith('buildings') or c.startswith('pop'))]
    print(key, df.index.size)
    
#     df0 = df[['year'] + exp_cols].groupby('year').max().reset_index()
#     dfs[f'{key}'] = df0
#     risk[f'{key}'] = pd.Series({c: loglin_trapz(df0[c]) for c in exp_cols})

    df0 = df[['year']+exp_cols].groupby('year').sum().sort_values(exp_cols[0]).reset_index()
    dfs[f'{key}_sum'] = df0
    risk[f'{key}_sum'] = pd.Series({c: loglin_trapz(df0[c]) for c in exp_cols})

In [ ]:
df_risk = pd.DataFrame(risk).T
df_risk['buildings_value_base']

In [ ]:
df_risk[['buildings_value_base', 'population_count_base']].rename(labs).to_csv('risk_base.csv')
df_adapt = df_risk.loc[['magObs_timeObs_sum'], :].rename(labs)
df_adapt.to_csv('risk_adapt.csv')
df_adapt.T

In [ ]:
labs = {
    'fullDep': 'full dependence',
    'magObs_timeObs': 'obs. dependence',
    'magInd_timeInd': 'full indep.',
    'magInd_timeObs': 'independence',
    'magObs_timeObs_sum': 'obs. dependence',
    'magInd_timeInd_sum': 'full indep.',
    'magInd_timeObs_sum': 'independence',
    'p': 'pluvial', 
    'qp': 'fluvial Pungwe', 
    'qb': 'fluvial Buzi', 
    'h': 'coastal', 
}
rps= [1,2,5,10,50,100,500]
ylabs = {
    'buildings_value': 'annual building damage [mil. USD]',
    'population_count': 'annual people exposed [-]'
}
legend_titles = {
    'buildings_value': '(EAD [mil. USD])',
    'population_count': '(EAAP [x 1000])'
}

In [ ]:
import matplotlib.pyplot as plt
exp0, factor = 'population_count', 1e3
exp0, factor = 'buildings_value', 1e6
fig, ax = plt.subplots(1,1, figsize=(6,4))
for key in ['qp', 'qb', 'h', 'p'][-4:]:
    risk0 = risk[key][f'{exp0}_base']/factor
    lab = f'{labs[key]} ({risk0:.2f})'
#     lab = f'{labs[key]}'
    ls = '-' if 'dep' in lab else '--'
    zorder = 1 if 'dep' in lab else 2
    df0 = dfs[key].copy()
    if 'rp' not in df0:
        df0 = df0.sort_values(f'{exp0}_base')
        df0['rp'] = (1/(1-np.arange(df0.index.size)/df0.index.size))
    (df0.set_index('rp')[f'{exp0}_base']/factor).plot(ls=ls, lw=1.5, ax=ax, label=lab, zorder=zorder)
ax.set_xlim([1, 500])
if 'build' in exp0:
    ax.set_ylim([0, 150])
else:
    ax.set_ylim([0, 120])
ax.set_xscale('log')
ax.set_xticks(rps)
ax.set_xticklabels(rps)
ax.set_ylabel(f'{ylabs[exp0]}')
ax.set_xlabel('return period [year]')
# ax.legend(title='driver scenario', loc='upper left')
ax.legend(title=f'driver scenario\n {legend_titles[exp0]}', loc='upper left')
ax.set_title('Univariate flood risk - base scenario')
plt.savefig(os.path.join('../FIGURES', f'risk_curve_{exp0}_single.png'), dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
for key in ['magObs_timeObs_sum', 'magInd_timeObs_sum', 'fullDep']:
    risk0 = risk[key][f'{exp0}_base']/factor
    lab = f'{labs[key]} ({risk0:.2f})'
    ls = '--' if 'full' in key else '-'
    lw = 3 if key == 'magObs_timeObs_sum' else 1.5
    df0 = dfs[key].copy()
    if 'rp' not in df0:
        df0 = df0.sort_values(f'{exp0}_base')
        df0['rp'] = (1/(1-np.arange(df0.index.size)/df0.index.size))
    (df0.set_index('rp')[f'{exp0}_base']/factor).plot(ls=ls, lw=lw, ax=ax, label=lab)
ax.set_xlim([1, 500])
if 'build' in exp0:
    ax.set_ylim([0, 300])
else:
    ax.set_ylim([0, 150])
ax.set_xscale('log')
ax.set_xticks(rps)
ax.set_xticklabels(rps)
ax.set_ylabel(f'{ylabs[exp0]}')
ax.set_xlabel('return period [year]')
legend = ax.legend(title=f'compound scenario\n {legend_titles[exp0]}', loc='upper left')#, fontsize='large')
ax.set_title('Compound flood risk - base scenario')
plt.savefig(f'../FIGURES/risk_curve_{exp0}_compound.png', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6, 4))
key='magObs_timeObs_sum'
for pf in ['base', 'dikes', 'dryproofing', 'zoning']:
    exp = f'{exp0}_{pf}'
    risk0 = risk[key][exp]/factor
    pf = 'dry-proofing' if pf == 'dryproofing' else pf
    lab = f'{pf} ({risk0:.2f})'
#     ls = '-' if pf == 'base' else ':'
    lw = 3 if pf == 'base' else 1.5
    df0 = dfs[key].copy()
    if 'rp' not in df0:
        df0 = df0.sort_values(exp)
        df0['rp'] = (1/(1-np.arange(df0.index.size)/df0.index.size))
    (df0.set_index('rp')[exp]/factor).plot(ls='-', lw=lw, ax=ax, label=lab)
ax.set_xlim([1, 500])
if 'build' in exp0:
    ax.set_ylim([0, 200])
else:
    ax.set_ylim([0, 140])
ax.set_xscale('log')
ax.set_xticks(rps)
ax.set_xticklabels(rps)
ax.set_ylabel(f'cumulative {ylabs[exp0]}')
ax.set_xlabel('return period [year]')
ax.legend(title=f'adaptation scenario\n {legend_titles[exp0]}', loc='upper left')
ax.set_title(f'Compound flood risk - {legend_titles[exp0][1:5]}')
plt.savefig(f'../FIGURES/risk_curve_{exp0}_adaptation.png', dpi=300, bbox_inches='tight')

In [ ]:
_df = samples['magObs_timeObs']
np.sum(np.logical_and.reduce((_df['qb_rp']>10, _df['qp_rp']>10, _df['h_rp']>10)))

### RISK MAPS

In [ ]:
ds_risk = (ds_impact * ds['cof']).sum('rps').compute()

# bias correct to match totals
df00 = df_adapt.T
df00 = df00.rename({n:n.replace('buildings_value', 'cp_dam').replace('_base', '') for n in df00.index})
df00 = df00.rename({n:n.replace('population_count', 'cp_pop').replace('_base', '') for n in df00.index})

for dvar in df00.index:
    ds_risk[dvar] = ds_risk[dvar]/ds_risk[dvar].sum().item()*df00.loc[dvar].item()

In [ ]:
ds_risk = ds_risk.raster.reproject_like(mod0.staticmaps, 'nearest')
for dvar in ds_risk.data_vars:
    ds_risk[dvar] = ds_risk[dvar].where(mod0.staticmaps['msk']>0, -9999)
    ds_risk[dvar].raster.set_nodata(-9999)
ds_risk.raster.set_crs(mod0.crs)
dir_out = os.path.join('../03_models', 'risk')
ds_risk.raster.to_mapstack(dir_out, compress='lzw')